<a href="https://colab.research.google.com/github/SanjaghRecommand/Recommend/blob/Foad/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pandas as pd
import tensorflow as tf
import torch


In [ ]:
df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/taskfullshare.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
# df2=df.sort_values('user').iloc[0:20000,]

In [ ]:
mininit= df.Init.min()
df['time']=df.Init.apply(lambda x: round((x- mininit)/2000000000))
df.drop(columns='Init', inplace=True)
df= df[df['user'] >5 ]
df3=df


In [ ]:
df

,service,user,time
40602,0,6,1
131622,92,6,27
619004,84,7,52
657275,116,7,54
407940,10,7,44
...,...,...,...
561494,156,10508,50
47362,80,10508,14
47060,117,10508,14
46984,0,10512,14


In [ ]:
seqservice , seqtime ,seqtarget  =[] ,[] ,[] 

for user in df.user.unique():
  user_data = df[df['user'] == user].sort_values('time')
  seqservice.append(user_data.service.values.tolist()[0:-1])
  seqtime.append(user_data.time.values.tolist()[0:-1])
  seqtarget.append(user_data.time.values.tolist()[1:])

  # print(user_data)
  # break
train_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservice), tf.keras.utils.pad_sequences(seqtime), tf.keras.utils.pad_sequences(seqtarget)]))

In [ ]:
#@title Create_train
def create_train_tfdata(train_feat_dict, train_target_tensor,    batch_size, buffer_size=None):
    """
    Create train tf dataset for model train input
    :param train_feat_dict: dict, containing the features tensors for train data
    :param train_target_tensor: np.array(), the training TARGET tensor
    :param batch_size: (int) size of the batch to work with
    :param buffer_size: (int) Optional. Default is None. Size of the buffer
    :return: (tuple) 1st element is the training dataset,
                     2nd is the number of steps per epoch (based on batch size)
    """
    if buffer_size is None:
        buffer_size = batch_size*50

    train_steps_per_epoch = len(train_target_tensor) // batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_dict,train_target_tensor)).cache()
    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
    train_dataset = train_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    return train_dataset, train_steps_per_epoch
  
train_feat_dict = {'item_id': train_dict['item_id'],'nb_days': train_dict['nb_days']}
train_target_tensor = train_dict['target'] 
train_dataset, train_steps_per_epoch = create_train_tfdata(train_feat_dict,  train_target_tensor, batch_size=512)
  

In [ ]:
#@title BuildModel
def build_model(max_len, item_vocab_size):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],   name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],  name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=50,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=100,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization(
     name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat'
                                   )(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(att)

    model = tf.keras.Model(inputs, output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.005),
        loss=loss_function,
        metrics=['sparse_categorical_accuracy'])
    
    return model

In [ ]:
#@title LossFun
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
#@title FitModel
def fit_model(model, train_dataset, steps_per_epoch, epochs):
    """
    Fit the Keras model on the training dataset for a number of given epochs
    :param model: tf model to be trained
    :param train_dataset: (tf.data.Dataset object) the training dataset
                          used to fit the model
    :param steps_per_epoch: (int) Total number of steps (batches of samples) before 
                            declaring one epoch finished and starting the next epoch.
    :param epochs: (int) the number of epochs for the fitting phase
    :return: tuple (mirrored_model, history) with trained model and model history
    """
        # mirrored_strategy allows to use multi GPUs when available
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        tf.distribute.experimental.CollectiveCommunication.AUTO)
    
    with mirrored_strategy.scope():
        mirrored_model = model

    history = mirrored_model.fit(train_dataset,  steps_per_epoch=steps_per_epoch,     epochs=epochs, verbose=2)

    return mirrored_model, history

In [ ]:
max_len= len(tf.keras.utils.pad_sequences(seqservice)[0])
item_vocab_size= 600

model=build_model(max_len, item_vocab_size)
fit_model(model, train_dataset,50, 35)

Epoch 1/35
50/50 - 13s - loss: 0.0602 - sparse_categorical_accuracy: 0.4038 - 13s/epoch - 265ms/step
Epoch 2/35
50/50 - 8s - loss: 0.0396 - sparse_categorical_accuracy: 0.4704 - 8s/epoch - 155ms/step
Epoch 3/35
50/50 - 8s - loss: 0.0345 - sparse_categorical_accuracy: 0.4800 - 8s/epoch - 155ms/step
Epoch 4/35
50/50 - 7s - loss: 0.0314 - sparse_categorical_accuracy: 0.4971 - 7s/epoch - 136ms/step
Epoch 5/35
50/50 - 8s - loss: 0.0301 - sparse_categorical_accuracy: 0.5042 - 8s/epoch - 151ms/step
Epoch 6/35
50/50 - 7s - loss: 0.0287 - sparse_categorical_accuracy: 0.5012 - 7s/epoch - 137ms/step
Epoch 7/35
50/50 - 7s - loss: 0.0268 - sparse_categorical_accuracy: 0.5126 - 7s/epoch - 144ms/step
Epoch 8/35
50/50 - 7s - loss: 0.0253 - sparse_categorical_accuracy: 0.5195 - 7s/epoch - 142ms/step
Epoch 9/35
50/50 - 7s - loss: 0.0242 - sparse_categorical_accuracy: 0.5542 - 7s/epoch - 146ms/step
Epoch 10/35
50/50 - 7s - loss: 0.0216 - sparse_categorical_accuracy: 0.5459 - 7s/epoch - 136ms/step
Epoch 1

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebook/sanjagh/Recommand/savedmodels/modelRNN")

In [ ]:
len(seqservice)

426022